In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import IPython.display as ipd
import pyximport
pyximport.install()
%load_ext Cython

import sigkernel as ksig
from utils.midi import *
from utils.data import *
from model.generators import *

In [2]:
hist_len = 50
sample_len = 100
seq_dim = 4
scale = 1.

In [3]:
path = './data/maestro-v3.0.0_midi/2004/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi'
midi_data = pretty_midi.PrettyMIDI(path)
df = midi_to_df(midi_data)
dataset = DFDataset(df, sample_len, scale, stride=20, col_idx=list(range(4)))
dataloader = DataLoader(dataset, batch_size=50, shuffle=True, num_workers=0)

In [4]:
for x in dataloader:
    print(x)
    break

tensor([[[0.0000e+00, 1.4169e-01, 5.0000e+01, 7.0000e+01],
         [1.8646e-01, 2.1875e-01, 5.7000e+01, 7.0000e+01],
         [1.9583e-01, 3.5834e-01, 7.9000e+01, 7.1000e+01],
         ...,
         [9.0844e+00, 9.2167e+00, 4.7000e+01, 6.7000e+01],
         [9.2365e+00, 9.4115e+00, 4.0000e+01, 6.6000e+01],
         [9.2510e+00, 9.2906e+00, 7.1000e+01, 7.0000e+01]],

        [[0.0000e+00, 1.6250e-01, 7.6000e+01, 7.9000e+01],
         [2.0104e-01, 4.6458e-01, 7.9000e+01, 7.6000e+01],
         [4.1979e-01, 4.8021e-01, 7.2000e+01, 7.0000e+01],
         ...,
         [1.2309e+01, 1.2561e+01, 7.5000e+01, 6.9000e+01],
         [1.2516e+01, 1.3089e+01, 5.2000e+01, 6.1000e+01],
         [1.2522e+01, 1.4464e+01, 7.6000e+01, 6.5000e+01]],

        [[0.0000e+00, 1.3541e-01, 7.9000e+01, 8.2000e+01],
         [2.0752e-03, 1.5938e-01, 5.2000e+01, 7.6000e+01],
         [1.2604e-01, 2.1875e-01, 8.1000e+01, 8.0000e+01],
         ...,
         [6.7469e+00, 6.8292e+00, 7.8000e+01, 6.2000e+01],
         [

In [5]:
generator = TransformerMusic(seq_dim, sample_len, hist_len, scale=scale,
                             kernel_size=5, stride=1, n_channels=16, n_head=4, n_transformer_layers=1, hidden_size=128, activation='GELU')
generator = generator.cuda()
optimizer = torch.optim.Adam(generator.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5, verbose=True)

In [6]:
static_kernel = ksig.static.kernels.RationalQuadraticKernel(sigma=0.1)
kernel = ksig.kernels.SignatureKernel(n_levels=5, order=5, normalization=0, static_kernel=static_kernel, device_ids=None)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for epoch in range(100):
    losses = [] # due to legacy code, losses is actually the mmd
    for batch_num, X in enumerate(tqdm(dataloader)):
        X = X.to(device)

        output = generator(X)
        X_wo_hist = X[:, hist_len:, :]

        # compute loss
        optimizer.zero_grad()
        loss = ksig.tests.mmd_loss_no_compile(X_wo_hist, output, kernel)
        losses.append(loss.item())

        # backpropagate and update weights
        loss.backward()
        optimizer.step()

    # log epoch loss and plot generated samples
    epoch_loss = np.average(losses) # average batch mmd for epoch
    scheduler.step(epoch_loss)
    print(f'Epoch {epoch}, loss: {epoch_loss}')

100%|██████████| 8/8 [00:12<00:00,  1.59s/it]


Epoch 0, loss: 0.00034290552139282227


100%|██████████| 8/8 [00:12<00:00,  1.53s/it]


Epoch 1, loss: 0.0002301335334777832


100%|██████████| 8/8 [00:12<00:00,  1.52s/it]


...

Epoch 71, loss: 0.0004452913999557495


100%|██████████| 8/8 [00:12<00:00,  1.51s/it]


Epoch 00073: reducing learning rate of group 0 to 1.5625e-05.
Epoch 72, loss: 0.0002434849739074707


 50%|█████     | 4/8 [00:07<00:07,  1.95s/it]


KeyboardInterrupt: 

In [8]:
for x in dataloader:
    x = x.to(device)
    output = generator(x)
    break

In [9]:
i = 0
x_np = x[i].cpu().detach().numpy()
x_np[:, 2:] = np.round(x_np[:, 2:] * scale)
output_np = output[i].cpu().detach().numpy()
output_np[:, 2:] = np.round(output_np[:, 2:] * scale)
df_x = pd.DataFrame(x_np, columns=['Start', 'End', 'Pitch', 'Velocity'])
df_output = pd.DataFrame(output_np, columns=['Start', 'End', 'Pitch', 'Velocity'])

In [10]:
df_input = df_x.iloc[hist_len:]
df_input.iloc[:,:2] = df_input.iloc[:,:2] - df_input.iloc[0,0]
df_input

/tmp/ipykernel_11415/2444262015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input.iloc[:,:2] = df_input.iloc[:,:2] - df_input.iloc[0,0]


,Start,End,Pitch,Velocity
50,0.000000,0.088562,62.0,78.0
51,0.020874,0.153137,79.0,64.0
52,0.037537,0.093750,77.0,70.0
53,0.136475,0.188538,67.0,74.0
54,0.146912,0.185425,84.0,61.0
55,0.152100,0.185425,83.0,67.0
56,0.276062,0.427124,60.0,76.0
57,0.277100,0.369812,76.0,83.0
58,0.414612,0.491699,64.0,66.0
59,0.433350,0.485413,79.0,57.0


In [11]:
df_output.iloc[:,:2] = df_output.iloc[:,:2] - df_output.iloc[0,0]
df_output

,Start,End,Pitch,Velocity
0,0.000000,0.785002,0.0,1.0
1,0.016623,0.172844,61.0,36.0
2,0.060726,1.117059,113.0,115.0
3,0.073077,2.074643,30.0,22.0
4,0.084586,1.184121,73.0,75.0
5,0.086188,0.339140,42.0,121.0
6,0.095097,2.512566,61.0,4.0
7,0.119560,0.799353,84.0,17.0
8,0.144665,1.263191,89.0,105.0
9,0.184938,0.583490,77.0,23.0


In [12]:
input_midi = df_to_midi(df_input)
output_midi = df_to_midi(df_output)

In [15]:
Fs = 22050
audio_data = input_midi.synthesize(fs=Fs)
ipd.Audio(audio_data, rate=Fs)

In [16]:
audio_data = output_midi.synthesize(fs=Fs)
ipd.Audio(audio_data, rate=Fs)